In [ ]:
import sqlite3
import pandas as pd
import numpy as np

engine = sqlite3.connect('Graph_DB')

graph_df = pd.read_sql('select * from graph_df', engine)

In [ ]:
sampled_df = graph_df[['sampled']]
sampler_df = graph_df[['sampler']]
sampler_df = sampler_df.rename(index=str, columns={'sampler':'sampled'})
all_ids = sampled_df.append(sampler_df, ignore_index = True)
all_ids = all_ids.drop_duplicates()
all_ids = all_ids.rename(index=str, columns={'sampled':'song'})
all_ids.to_sql('songs_df', engine, if_exists='replace', index=True)

In [ ]:
auth_df = pd.read_sql('select song, 1 as auth from songs_df', engine)
hub_df = pd.read_sql('select song, 1 as hub from songs_df', engine)
auth_df.to_sql('auth_db', engine, if_exists='replace', index=True)
hub_df.to_sql('hub_db', engine, if_exists='replace', index=True)

In [ ]:
num_iter = 10
for i in range(0, num_iter):
    hub_l2 = np.sqrt((hub_df['hub']**2).sum())
    auth_l2 = np.sqrt((auth_df['auth']**2).sum())
    hub_df = pd.read_sql('select song, SUM(auth) / ' + str(auth_l2) +' as hub from graph_df left join auth_db on ' +
                         'auth_db.song = graph_df.sampler group by song', engine)
    auth_df = pd.read_sql('select song, SUM(hub) / ' + str(hub_l2) +' as auth from graph_df left join hub_db on ' +
                         'hub_db.song = graph_df.sampled group by song', engine)
    auth_df.to_sql('auth_db', engine, if_exists='replace', index=True)
    hub_df.to_sql('hub_db', engine, if_exists='replace', index=True)

In [ ]:
hub_df.sort_values(by=['hub'], ascending=0).head(10)

In [ ]:
auth_df.sort_values(by=['auth'], ascending=0).head(10)